<img src="https://www.haskell.org/static/img/haskell-logo.svg?etag=ukf3Fg7-">

# Haskell Brooks Curry

<img src="https://upload.wikimedia.org/wikipedia/commons/8/86/HaskellBCurry.jpg">

(Photo from Wikipedia)

# Tuples

In [1]:
tuple = (0, 1)
fst tuple
snd tuple

0

1

# Lists

In [2]:
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
[1 .. 10]
[1 .. 5] ++ [6 .. 10]

[1,2,3,4,5,6,7,8,9,10]

[1,2,3,4,5,6,7,8,9,10]

[1,2,3,4,5,6,7,8,9,10]

In [3]:
head [1 .. 10]
tail [1 .. 10]

1

[2,3,4,5,6,7,8,9,10]

In [4]:
-- pattern-matching
hd [] = error "" -- assertion failure
hd (x: _) = x
tl [] = error "" -- assertion failure
tl (_: x) = x
hd [1 .. 10]
tl [1 .. 10]

1

[2,3,4,5,6,7,8,9,10]

In [5]:
filter (> 5) [1 .. 10] -- (> 5) is currying
map (> 5) [1 .. 10]
foldl (+) 0 [1 .. 10]
foldr (+) 0 [1 .. 10]

Line 1: Use sum
Found:
foldl (+) 0
Why not:
sumLine 1: Use sum
Found:
foldr (+) 0
Why not:
sum

[6,7,8,9,10]

[False,False,False,False,False,True,True,True,True,True]

55

55

# Strings

In [6]:
:t "abc"
['a' .. 'z']

"abc" :: [Char]

"abcdefghijklmnopqrstuvwxyz"

# Functions

In [7]:
:t div -- typing

div :: forall a. Integral a => a -> a -> a

In [8]:
-- composition of functions
sin (cos 1)
(sin . cos) 1

0.5143952585235492

0.5143952585235492

In [9]:
-- application of function
sin (cos 1)
sin $ cos 1 -- $, $!, `seq` are operators of lowest precedence in Haskell

0.5143952585235492

0.5143952585235492

In [10]:
-- prefix & infix

div 1 1 -- integer division
1 `div` 1
1 / 1
(/) 1 1

1

1

1.0

1.0

# $\lambda$-abstractions

In [11]:
quadI = \x y -> (x > 0) && (y > 0)
quadI 1 1

Line 1: Redundant lambda
Found:
quadI = \ x y -> (x > 0) && (y > 0)
Why not:
quadI x y = (x > 0) && (y > 0)

True

# User-defined data structures

In [12]:
data TheThreeMusketeers = Athos | Aramis | Porthos
Athos

No instance for (Show TheThreeMusketeers) arising from a use of `print'
In a stmt of an interactive GHCi command: print it

In [13]:
-- deriving Show
data TheThreeMusketeers = Athos | Aramis | Porthos deriving Show
Athos

Athos

In [14]:
-- different constructors for a type
data Point = Cartesian Double Double | Polar Double Double deriving Show
p = Polar (pi / 2) 1
toCartesian (Polar theta rho) = Cartesian (rho * cos theta) (rho * sin theta)
toCartesian p

Cartesian 6.123233995736766e-17 1.0

In [15]:
-- deriving Read
data Cartesian = Cartesian Double Double deriving (Show, Read)
reads "Cartesian 1.0 1.0 lies in Quadrant I." :: [(Cartesian, String)]

[(Cartesian 1.0 1.0," lies in Quadrant I.")]

In [16]:
-- deriving Enum
data TheThreeMusketeers = Athos | Aramis | Porthos deriving (Show, Enum)
fromEnum Athos
enumFrom Athos
[Athos ..]

0

[Athos,Aramis,Porthos]

[Athos,Aramis,Porthos]

In [17]:
-- deriving (Eq, Ord)
data TheThreeMusketeers = Athos | Aramis | Porthos deriving (Show, Eq, Ord)
Athos == Aramis
Athos < Aramis -- lexicographical order

False

True

# Parameterized and recursive types

In [18]:
data List t = Cons t (List t) | Nil deriving Show
Cons Athos (Cons Aramis (Cons Porthos Nil))

Cons Athos (Cons Aramis (Cons Porthos Nil))

In [19]:
hd Nil = error ""
hd (Cons x _) = x
tl Nil = error ""
tl (Cons _ xs) = xs
hd $ Cons Athos (Cons Aramis (Cons Porthos Nil))
tl $ Cons Athos (Cons Aramis (Cons Porthos Nil))

Athos

Cons Aramis (Cons Porthos Nil)

# Recursive binding

There is **only** recursive binding in Haskell.

In [20]:
x = 0
let x = x + 1
-- Believe me, it will not terminate.
-- x

In [21]:
-- use "variable shadowing" to simulate non-recursive binding
x = 0
let y = x
let x = y + 1
x

1

# Tail recursion

In [22]:
-- an implementation that is O(n) in space
factorial 0 = 1
factorial n = n * factorial (n - 1)
factorial 3

6

In [23]:
-- tail-recursive implementation (essentially a loop)
factorial n = let loop fac n' = if n' == 0 then fac else loop (fac * n') (n' - 1) in loop 1 n
factorial 3

6

# Laziness

In [24]:
-- This will terminate.
x = 0
let x = x + 1

In [25]:
-- nonnegative extended real number system
data ER = R Double | Inf deriving Show

en_div Inf Inf = error "undefined"
en_div Inf _ = Inf
en_div _ Inf = R 0
en_div (R x) (R y) = let q = x / y in
                     if y == 0.0 then Inf else (R q)
zero = R 0
one = R 1
en_div one zero

Inf

# seq

Seq is defined by:

$seq\ \perp\ x = \perp$

and

$seq\ x\ y = y$,

where $\perp$ refers to unsuccessful computation.

It introduces **strictness**.

Note: if the compiler manages to prove statically that $x$ must be successful, it is **unnecessary** for the program to evaluate $x$ before evaluating $y$ (which is generally impossible).

In [26]:
-- Although factorial is tail-recursive, without GHC optimization it can create O(n) "thunks".
factorial n = let l f n' = if n' == 0 then f else l (f * n') (n' - 1) in l 1 n

In [27]:
-- use seq to "force evaluation"

-- `seq` is the infix version of seq
factorial n = let l f n' = if n' == 0 then f else f `seq` l (f * n') (n' - 1) in l 1 n

-- shorthand
factorial n = let l f n' = if n' == 0 then f else (l $! (f * n')) (n' - 1) in l 1 n

# Expression vs. declaration style

In [28]:
-- quick sort in expression style
qsort [] = []
qsort (x: xs) =
    let lt = filter (< x) xs in
    let geq = filter (>= x) xs in
    (qsort lt) ++ [x] ++ (qsort geq)

In [29]:
-- quick sort in declaration style
qsort [] = []
qsort (x: xs) = (qsort lt) ++ [x] ++ (qsort geq) 
                    where lt = filter (< x) xs
                          geq = filter (>= x) xs